In [3]:
import yfinance as yf
import pandas as pd
import numpy as np
from itertools import combinations
import warnings
warnings.filterwarnings('ignore')

# 28 Major FX pairs (Yahoo Finance tickers)
FX_PAIRS = [
    'EURUSD=X', 'GBPUSD=X', 'USDJPY=X', 'USDCHF=X', 'AUDUSD=X', 'USDCAD=X', 'NZDUSD=X',
    'EURGBP=X', 'EURJPY=X', 'EURCHF=X', 'EURCAD=X', 'EURAUD=X', 'EURNZD=X',
    'GBPJPY=X', 'GBPCHF=X', 'GBPCAD=X', 'GBPAUD=X', 'GBPNZD=X',
    'AUDJPY=X', 'AUDCHF=X', 'AUDCAD=X', 'AUDNZD=X',
    'NZDJPY=X', 'NZDCHF=X', 'NZDCAD=X', 'CADJPY=X', 'CADCHF=X', 'CHFJPY=X'
]


def fetch_fx_data(pairs, period='10d', interval='15m'):
    """Fetch OHLC data for FX pairs"""
    data = yf.download(pairs, period=period, interval=interval)['Close']
    data = data.dropna(axis=1, thresh=len(data)*0.8)  # Drop pairs with too many NaNs
    return data

def calculate_log_returns(prices):
    """Calculate log returns: ln(Pt/Pt-1)"""
    return np.log(prices / prices.shift(1)).dropna()

def correlation_matrix(returns):
    """Compute full correlation matrix"""
    return returns.corr()

def detect_decoupling_signals(pairs, high_corr_threshold=0.5, drop_threshold=0.1):
    """
    Detect pairs with high 10-day correlation that dropped recently (24h)
    Returns: list of (pair1, pair2, corr_10d, corr_24h)
    """
    # 10-day M15 data
    prices_10d = fetch_fx_data(pairs, period='10d', interval='15m')  # Limit for speed
    returns_10d = calculate_log_returns(prices_10d)
    corr_10d = correlation_matrix(returns_10d)
    
    # 24-hour M15 data  
    prices_24h = fetch_fx_data(pairs, period='1d', interval='15m')
    returns_24h = calculate_log_returns(prices_24h)
    corr_24h = correlation_matrix(returns_24h)
    
    signals = []
    for pair1, pair2 in combinations(prices_10d.columns, 2):
        corr_long = corr_10d.loc[pair1, pair2]
        corr_short = corr_24h.loc[pair1, pair2] if pair1 in corr_24h.index and pair2 in corr_24h.columns else np.nan
        
        if (abs(corr_long) > high_corr_threshold and 
            not np.isnan(corr_short) and 
            abs(corr_short) < (abs(corr_long) - drop_threshold)):
            signals.append((pair1, pair2, corr_long, corr_short))
    
    return signals

# RUN ANALYSIS
print("=== FULL CORRELATION MATRIX (10 days M15) ===")
prices = fetch_fx_data(FX_PAIRS, period='10d', interval='15m')
returns = calculate_log_returns(prices)
corr_matrix = correlation_matrix(returns)
print(corr_matrix.round(3))

print("\n=== DECOUPLING SIGNALS (Hedging Opportunities) ===")
signals = detect_decoupling_signals(FX_PAIRS)
for signal in signals:
    p1, p2, c10d, c24h = signal
    print(f"{p1} vs {p2}: 10d={c10d:.3f} → 24h={c24h:.3f} (DROP: {c10d-c24h:.3f})")


=== FULL CORRELATION MATRIX (10 days M15) ===


[*********************100%***********************]  28 of 28 completed


Ticker    AUDCAD=X  AUDCHF=X  AUDJPY=X  AUDNZD=X  AUDUSD=X  CADCHF=X  \
Ticker                                                                 
AUDCAD=X     1.000     0.541     0.484     0.242     0.610    -0.009   
AUDCHF=X     0.541     1.000     0.581     0.298     0.513     0.494   
AUDJPY=X     0.484     0.581     1.000     0.322     0.578     0.316   
AUDNZD=X     0.242     0.298     0.322     1.000     0.140     0.118   
AUDUSD=X     0.610     0.513     0.578     0.140     1.000     0.085   
CADCHF=X    -0.009     0.494     0.316     0.118     0.085     1.000   
CADJPY=X    -0.114     0.300     0.724     0.182     0.116     0.521   
CHFJPY=X    -0.002    -0.143     0.501     0.091     0.035    -0.137   
EURAUD=X    -0.539    -0.623    -0.696    -0.261    -0.694    -0.249   
EURCAD=X     0.356    -0.179    -0.220    -0.050    -0.012    -0.504   
EURCHF=X     0.183     0.506     0.225     0.111     0.107     0.473   
EURGBP=X    -0.017    -0.063    -0.002     0.057    -0.112    -0

[*********************100%***********************]  28 of 28 completed
[*********************100%***********************]  28 of 28 completed

AUDUSD=X vs EURUSD=X: 10d=0.545 → 24h=0.389 (DROP: 0.156)
AUDUSD=X vs GBPUSD=X: 10d=0.566 → 24h=0.438 (DROP: 0.128)
CADJPY=X vs GBPCAD=X: 10d=-0.511 → 24h=-0.366 (DROP: -0.146)
EURCHF=X vs GBPCHF=X: 10d=0.626 → 24h=0.480 (DROP: 0.146)
GBPNZD=X vs NZDCAD=X: 10d=-0.565 → 24h=-0.352 (DROP: -0.213)
GBPNZD=X vs NZDJPY=X: 10d=-0.594 → 24h=-0.401 (DROP: -0.193)
GBPNZD=X vs NZDUSD=X: 10d=-0.632 → 24h=-0.516 (DROP: -0.116)
